In [1]:
import time
import copy

import pandas as pd
import geopandas as gpd
import spatial_network_analysis as sna

import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint, LineString, Polygon
import momepy
import networkx as nx

# Create Graph - DONE
# define projected crs 

In [6]:
gdf_streets.head()

,id,codigo,nomoficial,alt_izqini,alt_izqfin,alt_derini,alt_derfin,nomanter,nom_mapa,tipo_c,...,tipo_ffcc,COMUNA,COM_PAR,COM_IMPAR,BARRIO,BARRIO_PAR,BARRIO_IMP,geometry,len_m,ttime_walking
0,0,16003,"OBLIGADO RAFAEL, Av.Costanera",6182,6200,6181,6199,"COSTANERA NORTE, Av.",AV.COSTANERA RAFAEL OBLIGADO,AVENIDA,...,NaN,13,13,13,PALERMO,PALERMO,PALERMO,"LINESTRING (-58.42922 -34.55085, -58.42969 -34...",56.40,0.705000
1,1,16003,"OBLIGADO RAFAEL, Av.Costanera",4202,4500,4201,4499,"COSTANERA NORTE, Av.",AV.COSTANERA RAFAEL OBLIGADO,AVENIDA,...,NaN,14,14,14,PALERMO,PALERMO,PALERMO,"LINESTRING (-58.41117 -34.55768, -58.41211 -34...",298.31,3.728875
2,2,19028,"LARRALDE, CRISOLOGO AV.",4202,4220,4201,4219,REPUBLIQUETAS,AV. CRISOLOGO LARRALDE,AVENIDA,...,Paso a Nivel,12,12,12,SAAVEDRA,SAAVEDRA,SAAVEDRA,"LINESTRING (-58.48435 -34.55768, -58.48464 -34...",30.60,0.382500
3,3,16003,"OBLIGADO RAFAEL, Av.Costanera",3452,4200,3451,4199,"COSTANERA NORTE, Av.",AV.COSTANERA RAFAEL OBLIGADO,AVENIDA,...,NaN,14,14,14,PALERMO,PALERMO,PALERMO,"LINESTRING (-58.40978 -34.55878, -58.41059 -34...",176.88,2.211000
4,4,21059,TRONADOR,0,0,0,0,ESTOMBA Y PLAZA,TRONADOR,CALLE,...,NaN,12,12,12,COGHLAN,COGHLAN,COGHLAN,"LINESTRING (-58.48099 -34.56048, -58.48105 -34...",9.41,0.117625


In [5]:
#gdf_streets = gpd.read_file("../2_Datos_Procesados/Calles_CII/Calles_topo.shp")
gdf_streets = gpd.read_file("data/CabaStreet_wgs84.zip")
gdf_streets = gdf_streets.explode().reset_index(drop=True)

gdf_streets = gdf_streets.to_crs("EPSG:5347") # ESTO PUEDE SER GENERICO DESDE UN CONFIG
gdf_streets['len_m'] = gdf_streets.length
gdf_streets.len_m = round(gdf_streets.len_m, 2)
gdf_streets = gdf_streets.to_crs("EPSG:4326")
gdf_streets['ttime_walking'] = gdf_streets['len_m'] / 80
gdf_streets['id'] = list(gdf_streets.index)

gdf_streets[['id', 'len_m', 'ttime_walking', 'geometry']].to_file("data/Calles_topo_timetravel.shp")

/tmp/ipykernel_802967/2096990861.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_streets = gdf_streets.explode().reset_index(drop=True)
/tmp/ipykernel_802967/2096990861.py:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_streets[['id', 'len_m', 'ttime_walking', 'geometry']].to_file("../data/Calles_topo_timetravel.shp")


DriverIOError: Failed to create file ../data/Calles_topo_timetravel.shp: No such file or directory

In [2]:
gdf, G, nodes, edges, mapping_nodes = sna.create_graph_from_file("../2_Datos_Procesados/Calles_CII/Calles_topo_timetravel.shp", "3857")
print(len(G.nodes), len(mapping_nodes['map_nodes']))

c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 125 disconnected components.
  warnings.warn(message)


66944 66944


# LOOP OVER LOCATIONS

In [3]:
gdf_locations = gpd.read_file("../1_Datos/EspaciosPublicos_v4.geojson", driver="GeoJSON")
gdf_locations = gdf_locations.to_crs("3857")
gdf_locations = gdf_locations.explode(index_parts=True).reset_index(drop=True)
gdf_locations = gdf_locations.loc[gdf_locations.evp == 1].reset_index(drop=True)
gdf_locations['id'] = list(gdf_locations.index +1)
gdf_locations.to_file("./data_output/locations.geojson", driver='GeoJSON')
# gdf_locations = gdf_locations.iloc[:5]
print(len(gdf_locations))
v_loc_type = 'polygon'

402


In [4]:
# error por suma de costo
""" print(len(G.nodes), len(mapping_nodes['map_nodes']))
gdf_locations_389 = gdf_locations.loc[(gdf_locations.id == 389)]
gdf_locations_389.to_file("./data_output/locations_389.geojson", driver='GeoJSON')
gdf_service_edges_389, gdf_service_locs_edges_389, gpd_all_service_areas_389 = sna.create_service_areas_for_locations(gdf_locations_389, 'polygon', G, nodes, edges, mapping_nodes, [5, 10, 15], 'ttime_walk', 20, 10, 200)
gdf_service_edges_389.to_file("./data_output/service_area_edges_389.geojson", driver='GeoJSON')
gdf_service_locs_edges_389.to_file("./data_output/service_area_locs_edges_389.geojson", driver='GeoJSON')
print(len(G.nodes), len(mapping_nodes['map_nodes'])) """

' print(len(G.nodes), len(mapping_nodes[\'map_nodes\']))\ngdf_locations_389 = gdf_locations.loc[(gdf_locations.id == 389)]\ngdf_locations_389.to_file("./data_output/locations_389.geojson", driver=\'GeoJSON\')\ngdf_service_edges_389, gdf_service_locs_edges_389, gpd_all_service_areas_389 = sna.create_service_areas_for_locations(gdf_locations_389, \'polygon\', G, nodes, edges, mapping_nodes, [5, 10, 15], \'ttime_walk\', 20, 10, 200)\ngdf_service_edges_389.to_file("./data_output/service_area_edges_389.geojson", driver=\'GeoJSON\')\ngdf_service_locs_edges_389.to_file("./data_output/service_area_locs_edges_389.geojson", driver=\'GeoJSON\')\nprint(len(G.nodes), len(mapping_nodes[\'map_nodes\'])) '

In [6]:
gdf_service_edges, gdf_service_locs_edges, gpd_all_service_areas = sna.create_service_areas_for_locations(gdf_locations, 'polygon', G, nodes, edges, mapping_nodes, [5, 10, 15], 'ttime_walk', 20, 10, 200)
gdf_service_edges.to_file("./data_output/service_area_edges.geojson", driver='GeoJSON')
gdf_service_locs_edges.to_file("./data_output/service_area_locs_edges.geojson", driver='GeoJSON')
gpd_all_service_areas.to_file("./data_output/service_areas.geojson", driver='GeoJSON')

Location id 189 not connected to network (d>20)
Location id 205 not connected to network (d>20)
Location id 219 not connected to network (d>20)
Location id 295 not connected to network (d>20)
Location id 296 not connected to network (d>20)


In [17]:
gpd_all_service_areas_rings = sna.dissolve_service_areas_polygons( gpd_all_service_areas, v_type='rings')
gpd_all_service_areas_rings.to_file("./data_output/service_area_rings.geojson", driver='GeoJSON')